In [35]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from time import sleep
import random

# headless webdriver
options = Options()
options.headless = True

sleepTimes = [2.1, 2.8, 3.2]

links = []
def page_actions():
    y = 1000
    # stop once try to scroll past page height
    while y <= driver.execute_script("return document.body.scrollHeight"):
        # slowly scroll down to load all products
        driver.execute_script("window.scrollTo(0, "+str(y)+")")
        y += 1000
        sleep(1)

    # get html
    soup = BeautifulSoup(driver.page_source,'html.parser')

    # for all product image links
    for a in soup.find_all(class_="gl-product-card__assets-link"):
        # bc of unisex overlap, check not a duplicate
        if a['href'] not in links:
            # add link to list
            links.append(a['href'])
        #if already in list, skip to next
        else:
            continue
    
    # if popup is present, click it so next can be clicked
    try:
        driver.find_element_by_class_name("gl-modal__close").click()
    except:
        pass

    
    try:
        # scroll so nav not in way of next page button
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000)")
        # click next page button
        driver.find_element_by_xpath('//*[@data-auto-id="pagination-right-button"]').click()
        # sleep random to look more human
        sleep(random.choice(sleepTimes))
        # repeat process
        page_actions()
    except:
        # if not found, on last page so finish
        pass

# open url, do function
url = "https://www.adidas.com/us/men-sustainable-apparel?start=0"
driver = webdriver.Firefox(options=options)
driver.get(url)
page_actions()
driver.quit()
print(len(links))

sleep(300)

# repeat for women
url = "https://www.adidas.com/us/women-sustainable-apparel"
driver = webdriver.Firefox(options=options)
driver.get(url)
page_actions()
driver.quit()
print(len(links))

1471
2710


2710

In [118]:
from IPython.display import clear_output
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.common.exceptions import TimeoutException

# GO BACK AND ADD THE FIRST ONE LATER ***************************************************** AND ADD MAKING A BLANK INFO_TEXT LIST

def list_actions(link_list):
    driver = webdriver.Firefox(options=options)

    for link in link_list:
        clear_output(wait=True)

        # some links don't include first part, if so, add it
        if re.match(r"^https://www.adidas.com", link) is None:
            driver.get("https://www.adidas.com" + link)
        else:
            driver.get(link)

        # wait until info div is present
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "navigation-target-specifications")))
        # if starts taking too long (more than 10 sec) and raises exception, quit and pause, change links_slice to start at next link (len of info_text since I ensured one list item per product link), function calls itself with new starting point
        except TimeoutException:
            driver.quit()
            sleep(300)
            links_slice = links[(len(info_text)):]
            list_actions(links_slice)

        soup = BeautifulSoup(driver.page_source,'html.parser')

        # find li, add to temporary list if include "%"
        li_list = []
        for li in soup.select('#navigation-target-specifications > div > div > ul > li'):
            if "%" in li.text:
                li_list.append(li.text)
            else:
                continue
        
        # join temp list to make sure if multiple pieces, stays in one list item
        li_list_join = " ".join(li_list)
        # add materials info to list
        info_text.append(li_list_join)
        
        # print progress
        print("slice restarted at", links.index(link_list[0]), "Current total progress:", round( ( (links.index(link) + 1) / len(links) )*100, 2 ), "%")
        # sleep to look more human
        sleep(random.choice(sleepTimes))
    
    driver.quit()

# kept getting timeouts bc list too long, instead of making so many slices, coded to edit starting point in function once get timeout exception
# initial restarting point (was 524 or 724)
links_slice = links[1988:]
list_actions(links_slice)

print(len(info_text))

slice restarted at 2387 Current total progress: 100.0 %


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51979): Max retries exceeded with url: /session/8b552eba-d4be-4b3d-b805-e50fca9de85b/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb31e0ce250>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [120]:
import pandas as pd

materials_adidas = pd.Series(info_text)
materials_adidas.to_csv("materials_adidas.csv")

not sure what happened with the blank ones....pretend they didn't happen lol